In [ ]:
# Imports

import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from ast import literal_eval
import os


In [ ]:
games = pd.read_csv('./data/games.csv')
recommendations = pd.read_csv('./data/recommendations.csv')
users = pd.read_csv('./data/users.csv')


In [ ]:
games.head(10)


In [ ]:
len(games)


In [ ]:
def getImagesURL(site, i):
    
    getpage_soup = BeautifulSoup(site.text, 'html.parser')
    images_tags = getpage_soup.findAll('a', {'class': 'highlight_screenshot_link'})
    
    images = []
    
    for img in images_tags:
        
        img = str(img)
        p1 = img.find('href')
        p2 = img.find(' rel')
        images.append(img[p1 + 6:p2 - 1])
        
    return images


In [ ]:
images_dataset = []

print("Total: ", len(games))

for i in range(len(games)):
    
    if (i % 1000 == 0):
        
        print('Now at: ', i)
        
        images = pd.DataFrame(images_dataset, columns = ['id', 'title', 'count', 'images_url'])
        images.to_csv('images_url_{}.csv'.format(int(i / 1000)), index = False)
        images_dataset = []
    
    site = 'https://store.steampowered.com/app/{}'.format(games.iloc[i].app_id)
    
    raw_site = requests.get(site)
    
    count = 0
    
    if (raw_site.status_code == 200):
        
        images = getImagesURL(raw_site, i)
        images_dataset.append([games.iloc[i].app_id, games.iloc[i].title, len(images), images])
    
    else:
        
        print('Error', raw_site)


In [ ]:
start = 3
end = 5

if not os.path.exists('./images'):
    
    os.mkdir('./images')
    
for i in range(start, end + 1):
    
    df = pd.read_csv('images_url_{}.csv'.format(int(i)))
    
    for j in range(len(df)):
        
        if (j % 100 == 0):
            
            print(j)
            
        if not os.path.exists('./images/{}'.format(df['id'][j])):
            
            os.mkdir('./images/{}'.format(df['id'][j]))
            
        images_links = literal_eval(df['images_url'][j])
        
        for k in range(len(images_links)):
            
            images_raw = requests.get(images_links[k])
            
            with open("./images/{}/{}.jpg".format(df['id'][j], k), "wb") as f:
                
                f.write(images_raw.content)
        